# First assignment

#### Packages and libraries installations

In [1]:
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 

Updating beautifulsoup4 is constricted by 
\ 
anaconda -> requires beautifulsoup4==4.7.1=py37_1

If you are sure you want an update of your package either try `conda update --all` or install a specific version of the package you want using `conda install <pkg>=<version>`

done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    anaconda-2019.03           |           py37_0          10 KB  anaconda
    beautifulsoup4-4.7.1       |           py37_1         143 KB  anaconda
    ca-certificates-2019.1.23 

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup



#### Importing, reading and checking data

In [3]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

In [4]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [5]:
df.shape

(288, 3)

### Grouping by Postcode first, aggregating neighbourhoods and adding the borough later

In [6]:
tt = df.groupby('Postcode', as_index=False).agg({'Neighbourhood':', '.join})
tt.head()

,Postcode,Neighbourhood
0,M1A,Not assigned
1,M1B,"Rouge, Malvern"
2,M1C,"Highland Creek, Rouge Hill, Port Union"
3,M1E,"Guildwood, Morningside, West Hill"
4,M1G,Woburn


In [7]:
tt['Borough'] = tt.Postcode.map(df.set_index('Postcode')['Borough'].to_dict())

In [8]:
tt.shape

(180, 3)

#### Rearranging the column indexes

In [9]:
tt = tt[['Postcode','Borough','Neighbourhood']]

#### This will drop all rows with not assigned Boroughs and Neighbourhoods

In [10]:
borind = tt[(tt['Borough'] == 'Not assigned')&(tt['Neighbourhood'] == 'Not assigned')].index

In [11]:
tt.drop(borind, inplace=True)

In [12]:
tt.head()

,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


#### Check for Not assigned Borough and Neighbourhoods

In [13]:
print(tt[(tt['Borough'] == 'Not assigned')].index.values)
print(tt[(tt['Neighbourhood'] == 'Not assigned')].index.values)

[]
[120]


One assigned Bourough contains one unassigned Neighbourhood that needs to be amended before proceeding  (Index 120)

#### Assigning Borough "cell" value to Neighbourhood not assigned "cell" value

In [14]:
tt['Neighbourhood'][(tt['Neighbourhood'] == 'Not assigned')] = tt['Borough'][(tt['Neighbourhood'] == 'Not assigned')]

#### Last check

In [15]:
print(tt[(tt['Borough'] == 'Not assigned')].index.values)
print(tt[(tt['Neighbourhood'] == 'Not assigned')].index.values)

[]
[]


In [16]:
tt.rename(columns={'Postcode':'PostalCode'}, inplace=True)
tt.reset_index(drop=True, inplace=True)

There is no presence of not assigned neighbourhoods or boroughs 

In [17]:
tt.head(11)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [18]:
tt.shape

(103, 3)

# Second assignment -------------------------------------------------------------------

In [19]:
!conda install -c conda-forge geocoder --yes

Solving environment: | 
  - anaconda::ca-certificates-2019.1.23-0
  - defaults::ca-certificates-2019.1.23done

## Package Plan ##

  environment location: /anaconda3

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    openssl-1.1.1b             |       h1de35cc_1         3.5 MB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         6.7 MB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim        

#### Using the Geospatial data file from cocl

In [23]:
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [24]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
tt.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [26]:
tt['Latitude'] = tt.PostalCode.map(geo.set_index('Postal Code')['Latitude'].to_dict())
tt['Longitude'] = tt.PostalCode.map(geo.set_index('Postal Code')['Longitude'].to_dict())

In [27]:
tt.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [28]:
tt.shape

(103, 5)

# Third assignment

In [29]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: \ 
  - anaconda::ca-certificates-2019.1.23-0
  - defaults::ca-certificates-2019.1.23done

## Package Plan ##

  environment location: /anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0



geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.20.0         | 57 KB     | ##################################### | 100% 
Preparing transaction: done

In [30]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: - 
  - anaconda::ca-certificates-2019.1.23-0
  - defaults::ca-certificates-2019.1.23done

## Package Plan ##

  environment location: /anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py37_0         773 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         872 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/osx-64::altair-3.2.0-py37_0
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             conda-forge/

#### Install libraries

In [31]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library


#### Get Toronto latitude and longitude

In [32]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create Map

In [33]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tt['Latitude'], tt['Longitude'], tt['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare credentials

In [34]:
CLIENT_ID = 'HJLPKNFYSMPOPHISTM0LKEJHZC1MK2ENDRP1RWT5NAWSYMCN' # your Foursquare ID
CLIENT_SECRET = 'G1I1EM0WQWTCR0JH0H334RF5FJMZMTJGXSLICV01B2P2MMMU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Definition of function to loop through the different neighbourhoods extracting data from foursquare

In [35]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Getting data from nearby venues

In [36]:
tv = getNearbyVenues(names=tt['Neighbourhood'], latitudes=tt['Latitude'], longitudes=tt['Longitude'])
backup = tv


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

#### One hot encoding

In [37]:
th = pd.get_dummies(tv[['Venue Category']], prefix="", prefix_sep="")
th.shape

(2266, 272)

#### Add neighborhood column back to dataframe

In [38]:
th['Neighbourhood'] = tv['Neighborhood'] 
th.shape

(2266, 273)

#### Move neighborhood column to the first column

In [39]:
# move neighborhood column to the first column
fixed_columns = [th.columns[-1]] + list(th.columns[:-1])
th = th[fixed_columns]

#### Grouping by neighbourhood by taking the mean of the frequency of occurrence

In [40]:
tg = th.groupby('Neighbourhood').mean().reset_index()
tg.shape

(99, 273)

#### Top venues per neighbourhood

In [41]:
num_top_venues = 5

for hood in tg['Neighbourhood']:
    print("----"+hood+"----")
    temp = tg[tg['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4            Hotel  0.03


----Agincourt----
                venue  freq
0        Skating Rink  0.17
1      Breakfast Spot  0.17
2              Lounge  0.17
3          Print Shop  0.17
4  Chinese Restaurant  0.17


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                        venue  freq
0                  Playground  0.33
1            Asian Restaurant  0.33
2                        Park  0.33
3  Modern European Restaurant  0.00
4                      Museum  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.25
1           Pizza Place  0.12
2              Pharmacy  0.12
3  Fast Food Restaurant  0.12
4            Beer Store  0.12


----Alderwood, Long Branch----
             

                        venue  freq
0      Furniture / Home Store   0.5
1              Baseball Field   0.5
2           Accessories Store   0.0
3  Modern European Restaurant   0.0
4               Movie Theater   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.14
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3   Sporting Goods Shop  0.03
4         Jewelry Store  0.03


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.12
1         Café  0.07
2        Hotel  0.04
3   Steakhouse  0.04
4   Restaurant  0.04


----Flemingdon Park, Don Mills South----
              venue  freq
0  Asian Restaurant  0.09
1        Beer Store  0.09
2       Coffee Shop  0.09
3               Gym  0.09
4         Bike Shop  0.05


----Forest Hill North, Forest Hill West----
                venue  freq
0       Jewelry Store  0.25
1    Sushi Restaurant  0.25
2  Mexican Restaurant  0.25
3               Trail  0.25
4   A

                venue  freq
0         Coffee Shop  0.11
1  Italian Restaurant  0.04
2          Restaurant  0.04
3                Café  0.04
4  Seafood Restaurant  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2               Bakery  0.05
3  American Restaurant  0.05
4   Italian Restaurant  0.05


----The Annex, North Midtown, Yorkville----
            venue  freq
0            Café  0.14
1  Sandwich Place  0.14
2     Coffee Shop  0.10
3    Burger Joint  0.05
4    Liquor Store  0.05


----The Beaches----
               venue  freq
0  Health Food Store  0.25
1                Pub  0.25
2              Trail  0.25
3       Neighborhood  0.25
4          Pet Store  0.00


----The Beaches West, India Bazaar----
            venue  freq
0  Sandwich Place  0.10
1     Pizza Place  0.10
2         Brewery  0.05
3             Pub  0.05
4      Steakhouse  0.05


----The Danforth West, Riverdale----
                    venue  freq
0 

#### Define function to get most common venues

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Apply function

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nvs = pd.DataFrame(columns=columns)
nvs['Neighbourhood'] = tg['Neighbourhood']

for ind in np.arange(tg.shape[0]):
    nvs.iloc[ind, 1:] = return_most_common_venues(tg.iloc[ind, :], num_top_venues)

nvs.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Bakery,Sushi Restaurant,Hotel,Asian Restaurant,American Restaurant
1,Agincourt,Lounge,Skating Rink,Sandwich Place,Chinese Restaurant,Print Shop,Breakfast Spot,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Asian Restaurant,Park,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Sandwich Place,Beer Store,Pizza Place,Gluten-free Restaurant,Deli / Bodega,Electronics Store
4,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Pharmacy,Coffee Shop,Pool,Pub,Sandwich Place,Skating Rink,Gym,American Restaurant


#### Run K-Means

In [45]:
# set number of clusters
kclusters = 5

tgc = tg.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tgc)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Add clustering labels

In [46]:
nvs.insert(0, 'Cluster Labels', kmeans.labels_)

tm = tt
tm = tm.join(nvs.set_index('Neighbourhood'), on='Neighbourhood')
tm = tm.drop('PostalCode', 1)

In [47]:
tm.head()


,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4.0,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Filipino Restaurant
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Pizza Place,Medical Center,Intersection,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
3,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
4,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Athletics & Sports,Bakery,Thai Restaurant,Caribbean Restaurant,Bank,Fried Chicken Joint,Discount Store,Doner Restaurant,Donut Shop


In [48]:
tm.shape

(103, 15)

#### Check for null values and transform doubles into integers for Cluster labels

In [49]:
tm['Cluster Labels'].isnull().values

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True, False, False, False, False,
       False, False, False, False])

There are some values missing in the Cluster Labels, we need to drop them

In [50]:
tm = tm.dropna()

In [51]:
tm.shape

(99, 15)

In [52]:
tm['Cluster Labels'] =tm['Cluster Labels'].astype(int)

#### Map creation

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tm['Latitude'], tm['Longitude'], tm['Neighbourhood'], tm['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters